<a href="https://colab.research.google.com/github/Rukaya-lab/Smart-Reply-suggest/blob/main/Smart_reply_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

     |████████████████████████████████| 11.0 MB 7.7 MB/s 
     |████████████████████████████████| 272 kB 95.7 MB/s 
     |████████████████████████████████| 2.4 MB 72.0 MB/s 
     |████████████████████████████████| 47 kB 6.9 MB/s 
     |████████████████████████████████| 2.5 MB 74.2 MB/s 
     |████████████████████████████████| 151 kB 94.6 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 506 kB 82.0 MB/s 
     |████████████████████████████████| 757 kB 89.0 MB/s 
     |████████████████████████████████| 1.0 MB 86.2 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=72104022ee03241c4a045bb7dcd3fc174f4b7dbde016a46936a3e3cda60ef9b9
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
Successfully built dill
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalle

     |████████████████████████████████| 6.8 MB 7.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
     |████████████████████████████████| 647 kB 6.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395159 sha256=5124e1ea75c901d242fc6158acbecfd4dbd917a0f83cf0b15cd1b966a715e7e0
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import string, os
import re
import random
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import annoy
from tensorflow.keras.preprocessing.text import Tokenizer
from scipy import spatial

In [ ]:
# reading dataset
df = pd.read_csv('/content/drive/MyDrive/topical_chat.csv')
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [ ]:
# dropping duplicates
df.drop_duplicates(['conversation_id', 'message'], inplace=True)
df.shape

(188336, 3)

In [ ]:

# preprocessing
def process(text):
    text = text.lower().replace('\n', ' ').replace('-', ' ').replace(':', ' ').replace(',', '') \
          .replace('"', '').replace("...", ".").replace("..", ".").replace("!", ".").replace("?", "").replace(";", ".").replace(":", " ")

    text = "".join(v for v in text if v not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii",'ignore')

    text = " ".join(text.split())
    return text


df.message = df.message.apply(process)

In [ ]:
df.head()

,conversation_id,message,sentiment
0,1,are you a fan of google or microsoft,Curious to dive deeper
1,1,both are excellent technology they are helpful...,Curious to dive deeper
2,1,im not a huge fan of google but i use it a lot...,Curious to dive deeper
3,1,google provides online related services and pr...,Curious to dive deeper
4,1,yeah their services are good im just not a fan...,Curious to dive deeper


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []

for conversation_index in tqdm(range(df.shape[0])):

    if conversation_index == 0:
        continue

    input_text = df.iloc[conversation_index - 1]
    target_text = df.iloc[conversation_index]


    if input_text.conversation_id == target_text.conversation_id:

        input_text = input_text.message
        target_text = target_text.message

        if len(input_text.split()) > 2 and \
            len(target_text.split()) > 0 and \
            len(input_text.split()) < 50 and \
            len(target_text.split()) < 10 and \
            input_text and \
            target_text:

            input_texts.append(input_text)
            target_texts.append(target_text)

100%|██████████| 188336/188336 [00:45<00:00, 4108.56it/s]


In [ ]:

print(len(input_texts))
print(len(target_texts))

21921
21921


In [ ]:
#saving
with open('input_texts.pickle', 'wb') as handle:
    pickle.dump(input_texts, handle, protocol=pickle.HIGHEST_PROTOCOL)

#saving
with open('target_texts.pickle', 'wb') as handle:
    pickle.dump(target_texts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

#tokenizing data

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)

In [ ]:
#saving
with open('input_tokenizer.pickle', 'wb') as handle:
    pickle.dump(input_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#saving
with open('target_tokenizer.pickle', 'wb') as handle:
    pickle.dump(target_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def generate_padded_sequences(sequences):
    max_sequence_len = max([len(x) for x in sequences])
    sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))
    return sequences, max_sequence_len

In [ ]:
input_total_words = len(input_tokenizer.word_index) + 1
input_sequences, input_max_sequence_len = generate_padded_sequences(input_sequences)

In [ ]:
print(input_max_sequence_len)

49


In [ ]:
# using annoy algorithm
from annoy import AnnoyIndex

inputAnnoyIndex = AnnoyIndex(input_max_sequence_len, 'angular')  # Length of item vector that will be indexed
for i, row in enumerate(input_sequences):
    inputAnnoyIndex.add_item(i, row)

inputAnnoyIndex.build(100) # 100 trees
inputAnnoyIndex.save('input_annoy.ann')

True

In [ ]:
query_index = 7
neighbor_index, distances = inputAnnoyIndex.get_nns_by_item(query_index, 10, include_distances=True)

print(input_texts[query_index])
print("\n-------------Nearest Neighbours-------------\n")
for i in range(len(neighbor_index)):
    print(input_texts[neighbor_index[i]], "--------- neighbor_index -", neighbor_index[i], ", distance -", distances[i])

hi do you like to dance

-------------Nearest Neighbours-------------

hi do you like to travel --------- neighbor_index - 10613 , distance - 0.028845567256212234
hi do you like the nba --------- neighbor_index - 13351 , distance - 0.05106751620769501
absolutely do you like to paint --------- neighbor_index - 2268 , distance - 0.06100903078913689
yes hes from not far from where im at the lakers also just signed michael beasley --------- neighbor_index - 11672 , distance - 0.09761030972003937
have you heard of the boy who came back from heaven --------- neighbor_index - 7830 , distance - 0.10321731865406036
i enjoyed the part when they stopped in at the little nightclub --------- neighbor_index - 1330 , distance - 0.12023560702800751
i have no idea who is it in order --------- neighbor_index - 12374 , distance - 0.12580275535583496
hi there do you like horses --------- neighbor_index - 15635 , distance - 0.14488904178142548
yes its interesting to know too that there are stars that can a

In [ ]:
import pickle
from annoy import AnnoyIndex

with open('/content/input_texts.pickle', 'rb') as handle:
  input_texts = pickle.load(handle)


inputAnnoyIndex = AnnoyIndex(49, 'angular')
inputAnnoyIndex.load('/content/input_annoy.ann')

True

In [ ]:
# generating similarity matrix for input texts
with open('input_similarity_matrix.txt', 'w') as input_similarity_matrix_file:
    for i in tqdm(range(len(input_texts))):
        neighbor_index, distances = inputAnnoyIndex.get_nns_by_item(i, len(input_texts), include_distances=True)

        input_similarity_row = [-1] * len(input_texts)
        for index in range(len(neighbor_index)):
            j = neighbor_index[index]
            input_similarity_row[j] = distances[index]


        input_similarity_matrix_file.write(str(input_similarity_row))
        if i != len(input_texts) - 1:
            input_similarity_matrix_file.write("\n")


input_similarity_matrix_file.flush()
input_similarity_matrix_file.close()

100%|██████████| 21921/21921 [1:14:20<00:00,  4.91it/s]


ValueError: ignored

In [ ]:

target_total_words = len(target_tokenizer.word_index) + 1
target_sequences, target_max_sequence_len = generate_padded_sequences(target_sequences)

targetAnnoyIndex = AnnoyIndex(target_max_sequence_len, 'angular')  # Length of item vector that will be indexed
for i, row in enumerate(target_sequences):
    targetAnnoyIndex.add_item(i, row)

targetAnnoyIndex.build(100) # 100 trees
targetAnnoyIndex.save('target_annoy.ann')

query_index = 7
neighbor_index, distances = targetAnnoyIndex.get_nns_by_item(query_index, 10, include_distances=True)

print(target_texts[query_index])
print("\n-------------Nearest Neighbours-------------\n")
for i in range(len(neighbor_index)):
    print(target_texts[neighbor_index[i]], "--------- neighbor_index -", neighbor_index[i], ", distance -", distances[i])

i love to dance a lot how about you

-------------Nearest Neighbours-------------

i love to dance a lot how about you --------- neighbor_index - 7 , distance - 0.0
sure i mostly like netflix how about you --------- neighbor_index - 8842 , distance - 0.09336796402931213
i know a little i guess how about you --------- neighbor_index - 12506 , distance - 0.10415032505989075
no i havent i heard it was good --------- neighbor_index - 1948 , distance - 0.10614748299121857
sure what kind of music do you like --------- neighbor_index - 14223 , distance - 0.16170845925807953
so true what kinds of entertainment do you like --------- neighbor_index - 6880 , distance - 0.17581990361213684
i have not watched that show is it good --------- neighbor_index - 9115 , distance - 0.186832457780838
you probably dont want to now well nice chatting --------- neighbor_index - 21334 , distance - 0.18750551342964172
they probably edited those parts out huh lol --------- neighbor_index - 10182 , distance - 0.18

In [ ]:
print(target_max_sequence_len)

9


In [ ]:
import pickle
from annoy import AnnoyIndex

with open('/content/target_texts.pickle', 'rb') as handle:
  target_texts = pickle.load(handle)


targetAnnoyIndex = AnnoyIndex(9, 'angular')
targetAnnoyIndex.load('/content/target_annoy.ann')

True

In [ ]:
# generating similarity matrix for target
with open('target_similarity_matrix.csv', 'w') as target_similarity_matrix_file:
    for i in tqdm(range(len(target_texts))):
        neighbor_index, distances = targetAnnoyIndex.get_nns_by_item(i, len(target_texts), include_distances=True)

        target_similarity_row = [-1] * len(target_texts)
        for index in range(len(neighbor_index)):
            j = neighbor_index[index]
            target_similarity_row[j] = distances[index]


        target_similarity_matrix_file.write(str(target_similarity_row))
        if i != len(target_texts) - 1:
            target_similarity_matrix_file.write("\n")


#target_similarity_matrix_file.flush()
target_similarity_matrix_file.close()

  6%|▋         | 1400/21921 [06:15<1:31:43,  3.73it/s]


KeyboardInterrupt: ignored